In [ ]:
%matplotlib inline

import pandas as pd
import dask.dataframe as dd
import dask
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import plotly as py
import psycopg2
import textatistic
import seaborn as sbn
import requests
matplotlib.style.use('ggplot')
py.offline.init_notebook_mode()
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_curve, auc

# Now with Dask

In [2]:
from dask.distributed import Client
#client = Client('192.203.196.71:8786')
#client = Client('127.0.0.1:8786')
client = Client('192.168.0.53:8786')

In [3]:
#ddata = dd.read_csv("../../Data/train_sample_clickfraud.csv", 
#                   parse_dates=['click_time','attributed_time'])

#dask.set_options(get=dask.multiprocessing.get)

ddata = dd.read_csv("/Users/bmiller/data/train.csv", 
                   parse_dates=['click_time','attributed_time'])



In [4]:
%%time

ddata = ddata.categorize(columns=["app","device","os","channel"])

CPU times: user 219 ms, sys: 29.3 ms, total: 248 ms
Wall time: 1min 58s


/Users/bmiller/anaconda3/lib/python3.6/site-packages/dask/dataframe/categorical.py:24: RuntimeWarning:

None of the categories were found in values. Did you mean to use
'Categorical.from_codes(codes, categories)'?



In [5]:
from dask_ml.preprocessing import DummyEncoder
from sklearn.linear_model import LogisticRegression

In [6]:
enc = DummyEncoder(["app","device","os","channel"])


In [7]:
ddata = enc.fit_transform(ddata)

In [8]:
from collections import OrderedDict

agg_funcs = OrderedDict()
for col in ddata.columns:
    agg_funcs[col] = 'sum'
    
agg_funcs['click_time'] = 'min'
agg_funcs['attributed_time']= 'min'


In [9]:
ddata = ddata.groupby('ip').agg(agg_funcs)

In [10]:
ddata.columns

Index(['ip', 'click_time', 'attributed_time', 'is_attributed', 'app_3',
       'app_14', 'app_64', 'app_18', 'app_6', 'app_58',
       ...
       'channel_127', 'channel_476', 'channel_434', 'channel_448',
       'channel_221', 'channel_490', 'channel_322', 'channel_404',
       'channel_474', 'channel_500'],
      dtype='object', length=5187)

In [11]:
ddata = ddata.assign(hour=ddata.click_time.dt.hour, minute=ddata.click_time.dt.minute, day=ddata.click_time.dt.day)

In [12]:
#ddata = ddata.assign(epoch=ddata.click_time.dt.)

In [13]:
X_train, X_test = ddata.random_split([0.8, 0.2], random_state=42)

In [14]:
y_train = X_train['is_attributed']
y_test = X_test['is_attributed']


In [15]:
X_train = X_train.drop(['is_attributed', 'attributed_time', 'click_time'], axis=1)
X_test = X_test.drop(['is_attributed', 'attributed_time','click_time'], axis=1)


In [16]:
from dask_ml.xgboost import XGBClassifier
dt_model = XGBClassifier()

In [17]:
y_train = y_train.map(lambda x: x if x == 0 else 1)

In [18]:
dt_model.fit(X_train, y_train)

distributed.utils - ERROR - ['tuple-186b2ebe-9786-4aa2-b4dc-1daa874e79a2']
Traceback (most recent call last):
  File "/Users/bmiller/anaconda3/lib/python3.6/site-packages/distributed/utils.py", line 238, in f
    result[0] = yield make_coro()
  File "/Users/bmiller/anaconda3/lib/python3.6/site-packages/tornado/gen.py", line 1099, in run
    value = future.result()
  File "/Users/bmiller/anaconda3/lib/python3.6/site-packages/tornado/gen.py", line 1107, in run
    yielded = self.gen.throw(*exc_info)
  File "/Users/bmiller/anaconda3/lib/python3.6/site-packages/dask_xgboost/core.py", line 108, in _train
    yield _wait(parts)
  File "/Users/bmiller/anaconda3/lib/python3.6/site-packages/tornado/gen.py", line 1099, in run
    value = future.result()
  File "/Users/bmiller/anaconda3/lib/python3.6/site-packages/tornado/gen.py", line 1113, in run
    yielded = self.gen.send(value)
  File "/Users/bmiller/anaconda3/lib/python3.6/site-packages/distributed/client.py", line 3239, in _wait
    raise 

CancelledError: ['tuple-186b2ebe-9786-4aa2-b4dc-1daa874e79a2']

In [21]:
#dt_model


In [19]:
dt_preds = dt_model.predict(X_test)

In [20]:
y_test = y_test.map(lambda x: x if x == 0 else 1)

In [21]:
confusion_matrix(y_test, dt_preds)

array([[   0, 6849],
       [   0,   46]])

In [25]:
from dask_ml.linear_model import PartialSGDClassifier

In [28]:
svm_model = PartialSGDClassifier(classes=[0,1])

In [45]:
svm_model.fit(X_train, y_train)

AttributeError: 'DataFrame' object has no attribute 'chunks'